## Hierarchical modularity in multiple cortical areas

We will collect neurons from several cortical areas to get the *functional connectivity* and perform the *dynamical analysis*.     
This will address three relevant points:
- Does the dynamic analysis hold at **higher temporal resolution**?
- Do **all areas of cortex** show attractor dynamics?
- Are population events only a **side-effect of behavior** (locomotion, whisker pad, pupil)?

To do all this, we analyse the [data](https://janelia.figshare.com/articles/dataset/Eight-probe_Neuropixels_recordings_during_spontaneous_behaviors/7739750/4) by [Stringer et al. 2019](science.org/doi/10.1126/science.aav7893).   
Eight-probe Neuropixels recordings in three mice during spontaneous activity.

In [6]:
from platform import python_version
print(python_version())

from builtins import exec
exec(open("./imports_functions.py").read())

%matplotlib inline

3.10.4


**WARNING**: the next cell takes time to download and unzip the neuropixel data.

In [9]:
if not os.path.exists("stringer/7739750.zip"):
    print("Downloading neuropixel data ...")
    resp = wget.download("https://janelia.figshare.com/ndownloader/articles/7739750/versions/4", "stringer/7739750.zip")
    print("... Done: "+resp)

# unzip downloaded folder
if os.path.exists("stringer/7739750.zip"):
    print("... unzipping")
    shutil.unpack_archive("stringer/7739750.zip", "stringer/7739750")
    shutil.unpack_archive("stringer/7739750/spks.zip", "stringer/7739750/spks")
    shutil.unpack_archive("stringer/7739750/faces.zip", "stringer/7739750/faces")
print("Done.")

... unzipping


### Data prepreocessing

This analysis is based on the file `ephysLoad.m`.

Each "spks" is a structure of length 8, where each entry is a different probe (these probes were recorded simultaneously). It contains the spike times (in seconds, e.g. 4048.44929626 sec (?kHz sampling)), the cluster identity of each spike (its cell), and the height of each cluster on the probe.

The location of each site on the probe in microns in the Allen CCF framework is given in "ccfCoords". The brain area for each site is in "borders" as a function of the height of the site. 

We need the spikes from each area and probe to be separate lists. So, we build a dictionary to hold them, and save it locally as `area_spiketrains.npy`.

In [2]:
if os.path.exists("stringer/7739750/area_spiketrains.npy"):
    area_spiketrains = np.load("stringer/7739750/area_spiketrains.npy", allow_pickle=True).item()
    print("... loaded populations")
else:
    print("... collecting populations")
    
    probeLoc = sio.loadmat('stringer/7739750/probeLocations.mat')
    probeBorders = sio.loadmat('stringer/7739750/probeBorders.mat', squeeze_me=True)

    mouse_names = ['Krebs','Waksman','Robbins']
    # mouse_names = ['Krebs']
    cortical_areas = ['FrCtx','FrMoCtx','SomMoCtx','SSCtx','V1','V2','RSP']

    # first count the cells you want to take with this structure
    # then think on how you want to store the spikes... compatible with the dynamical_analysis
    area_spiketrains = {
        'Krebs' : {'FrCtx':[], 'FrMoCtx':[], 'SomMoCtx':[], 'SSCtx':[], 'V1':[], 'V2':[], 'RSP':[]},
        'Waksman' : {'FrCtx':[], 'FrMoCtx':[], 'SomMoCtx':[], 'SSCtx':[], 'V1':[], 'V2':[], 'RSP':[]},
        'Robbins' : {'FrCtx':[], 'FrMoCtx':[], 'SomMoCtx':[], 'SSCtx':[], 'V1':[], 'V2':[], 'RSP':[]}
    }

    # start of spontaneous activity in each mouse (in seconds)
    tstart = [3811, 3633, 3323] # what's in there before?

    for imouse in range(len(mouse_names)):
        print(mouse_names[imouse])

        spks = sio.loadmat('stringer/7739750/spks/spks%s_Feb18.mat'%mouse_names[imouse], squeeze_me=True)

        # probe k
        # k = 7
        for k in range(8):
            print("probe",k)

            # spike times (in seconds)
            st = spks['spks'][k][0]
            # clusters
            clu = spks['spks'][k][1]
            print("clusters (cells) of the spikes",len(np.unique(clu)))
            # cluster heights (in microns)
            # (see siteCoords to convert to site location)
            Wh = spks['spks'][k][2]

            # where is the probe in the brain (consolidated labels)
            # borders are in microns
            # use Wh to determine which clusters are in which brain region
            borders = probeBorders['probeBorders'][imouse]['borders'][k]
            for j in range(len(borders)):
                population = [] # one population per border, there can be several borders
                b = borders[j]
                if b[2] not in cortical_areas:
                    continue
                print('upper border %d um, lower border %d um, area %s'%(b[0],b[1],b[2]))
                wneurons = np.logical_and(Wh>=b[1], Wh<b[0])
                nn = wneurons.sum()
                print('%d neurons in %s'%(nn,b[-1]))
                # we should not include population smaller than those in MICrONS
                if nn<10:
                    print('population too small. Rejected.')
                    continue

                cortical_neurons = np.nonzero(wneurons)[0]
                for cn in cortical_neurons:
                    cn_idxs = [i for i in range(len(clu)) if clu[i]==cn]
                    # print(cn_idxs)
                    population.append( sorted(st[cn_idxs]) )
                    
                area_spiketrains[ mouse_names[imouse] ][ b[2] ].append( population )
            print()

    # save to file
    np.save("stringer/7739750/area_spiketrains.npy", area_spiketrains)


... loaded populations


- **Krebs**
    - probe 0      
      upper border 4000 um, lower border 1100 um, area FrMoCtx      
      5 neurons in FrMoCtx
      population too small. Rejected.
    - probe 1      
      upper border 4000 um, lower border 1800 um, area FrMoCtx      
      73 neurons in FrMoCtx
    - probe 2      
      upper border 4000 um, lower border 2600 um, area V1      
      61 neurons in V1
    - probe 3      
        upper border 4000 um, lower border 2400 um, area V1      
        141 neurons in V1
    - probe 4      
        upper border 4000 um, lower border 1800 um, area SomMoCtx      
        65 neurons in SomMoCtx
    - probe 5      
        upper border 4000 um, lower border 2100 um, area SomMoCtx      
        26 neurons in SomMoCt
    - probe 6      
        upper border 4000 um, lower border 2350 um, area V1      
        68 neurons in V1
    - probe 7      
        upper border 4000 um, lower border 2600 um, area V1      
        64 neurons in V1


- **Waksman**
    - probe 0      
        upper border 4000 um, lower border 1700 um, area FrMoCtx      
        446 neurons in FrMoCtx      
        upper border 1200 um, lower border 0 um, area FrMoCtx      
        201 neurons in FrMoCtx
    - probe 1      
        upper border 4000 um, lower border 2150 um, area FrCtx      
        31 neurons in FrCtx
    - probe 2            
        upper border 4000 um, lower border 2700 um, area V1      
        155 neurons in V1
    - probe 3      
        upper border 4000 um, lower border 2250 um, area RSP      
        112 neurons in RSP
    - probe 4      
        upper border 4000 um, lower border 2000 um, area SomMoCtx      
        220 neurons in SomMoCtx
    - probe 5      
        upper border 4000 um, lower border 2600 um, area SSCtx      
        50 neurons in SSCtx
    - probe 6      
        upper border 4000 um, lower border 2650 um, area V2      
        124 neurons in V2
    - probe 7      
        upper border 4000 um, lower border 2850 um, area V1      
        96 neurons in V1     


- **Robbins**
    - probe 0      
        upper border 4000 um, lower border 3400 um, area FrMoCtx      
        16 neurons in FrMoCtx
    - probe 1      
        upper border 4000 um, lower border 3100 um, area FrMoCtx      
        70 neurons in FrMoCtx
    - probe 3      
        upper border 4000 um, lower border 3550 um, area RSP      
        10 neurons in RSP
    - probe 4      
        upper border 4000 um, lower border 3500 um, area SomMoCtx      
        10 neurons in SomMoCtx

In [3]:
real_frame_duration = 0.00000001 # sec (e.g. 4048.44929626 s)
frame_duration = 0.001 # sec (e.g. 4048.449 s)
local_path = os.getcwd() + '/stringer/7739750/'

In [7]:
for mousename,areas in area_spiketrains.items():
    print(mousename)
    exp_path = local_path + '%s/'%mousename
    
    for area,probe_populations in areas.items():
        if len(probe_populations)>0:
            print(area)
            
            for ipop,spiketrains in enumerate(probe_populations):
                
                # rounding to ms
                spiketrains = [np.round(sp, 3) for sp in spiketrains]
                start_time = min([min(st) for st in spiketrains])
                stop_time = max([max(st) for st in spiketrains])
                time = np.arange(start_time,stop_time,frame_duration)
                print("n_frames:",len(time)) # Krebs, FrMoCtx, n_frames: 1292157

                fig = plt.figure(figsize=[12.8,4.8])
                for row,train in enumerate(spiketrains):
                    plt.scatter( train, [row]*len(train), marker='o', edgecolors='none', s=1, c='k' )
                plt.ylabel("cell IDs")
                plt.xlabel("time (s)")
                # plt.show()
                fig.savefig(exp_path+'%s_%s_rasterplot.png'%(area,ipop), transparent=False, dpi=800)
                plt.tight_layout()
                plt.close()

                ophys_cell_ids = list(range(len(spiketrains)))
                ophys_cell_indexes = ophys_cell_ids # here is an alias

                global_events_sec = []
                global_events_duration = []
                global_cluster_number = []
                global_cluster_selfsimilarity = []
                global_clustered_spectrums = []

                core_reproducibility_perc = 60 # threshold for detecting cores
                scan_spiketrains = spiketrains
                scan_id = '_%s_%s'%(area,ipop)
                
                %run -e "dynamical_analysis.ipynb"

                global_events_sec.append(events_sec)
                global_events_duration.extend(events_durations_f)
                global_cluster_number.append(nclusters)
                global_cluster_selfsimilarity.extend(reproducibility_list)

    print()

Krebs
FrMoCtx
n_frames: 1292157
    population firing: 0.28±0.58 sp/frame
    cells firing rate: 0.00±0.06 sp/s
... generating surrogates to establish population event threshold
... loaded surrogates
    event size threshold (mean): 0.010399730569735933
... find peaks
... find minima
... find population events
... not enough events to continue analysis (<4)


TypeError: ipy_exit() takes 0 positional arguments but 1 was given

TypeError: ipy_exit() takes 0 positional arguments but 1 was given

In [ ]:
# # processed faces, to correlate behavior
# # The behavioral file is the processed version of a mouse face movie (time x pixels x pixels). 
# faces = io.loadmat('stringer/7739750/faces/%s_face_proc.mat'%mouse_names[imouse], squeeze_me=True)
# motSVD = faces['motionSVD']
# video_timestamps = faces['times']